In [ ]:
from quspin.operators import hamiltonian
from quspin.basis import spin_basis_1d
from quspin.tools.measurements import ent_entropy, diag_ensemble
from numpy.random import ranf,seed
from joblib import delayed,Parallel
import pyqentangle
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import svd
import math
L = 12
sizeA = 5
sizeB = 7
basis = spin_basis_1d(L,pauli=False)

def generate_Hamiltonian(v,g,norm = 2,J=1,h=1):
    u = np.sqrt(norm-v**2)
    Jx =  J*((u+v)/2)
    Jy = J*((v-u)/2)
    Jz = J*v
    J_xx = [[Jx, i,i+1] for i in range(L-1)]
    J_yy = [[Jy, i, i+1] for i in range(L-1)]
    J_zz = [[Jz, i, i+1] for i in range(L-1)]
    h_z = np.random.uniform(-h, h, size=L)
    h_x = np.random.uniform(-h, h, size=L)
    disordered_z_field = [[g * h_z[i], i] for i in range(L)]
    disordered_x_field = [[g * h_x[i], i] for i in range(L)]
    static = [["xx",J_xx], ["yy",J_yy], ["zz", J_zz], ["x", disordered_x_field], ["z", disordered_z_field]]
    dynamic = []
    no_checks = {"check_symm": False, "check_herm": False, "check_pcon": False}
    return hamiltonian(static, dynamic, basis = basis, dtype = np.float64, **no_checks)

def generate_random_state(L):
    random_state = np.random.normal(size=2) + 1.j * np.random.normal(size=2)
    for i in range(1,L):
        temp_state = np.random.normal(size=2) + 1.j * np.random.normal(size=2)
        temp_state /= np.linalg.norm(temp_state)
        random_state = np.kron(random_state, temp_state)
    return random_state

def generate_initial_state(L):
    init_psi = np.random.normal(size=2**L) + 1.j * np.random.normal(size=2**L)
    init_psi /= np.linalg.norm(init_psi)
    return init_psi
        
def entanglement_entropy(Hamiltonian, state, tmax, steps):
    times = np.linspace(0, tmax, steps)
    init_state = state
    psi = Hamiltonian.evolve(init_state, 0,times, eom="SE")
    step = 0
    entropy_values = []
    for i in range(len(times)):
        entropy_value = basis.ent_entropy(psi.T[i], sub_sys_A=range(L//2))["Sent_A"]
        entropy_values.append(entropy_value)
    return times, entropy_values

times = np.linspace(0,1,30)
H = generate_Hamiltonian(1,0.5)
init_state = generate_random_state(L)

x_values, y_values = entanglement_entropy(H,init_state, 350,350)
plt.plot(x_values, y_values, color='red', label='graph')
plt.show()

    
